# Dataset Discovery and Exploration: State-of-the-art, Challenges and Opportunities
## Part 1: Dataset Search
### Framework Overview -- D3L


This project utilizes structured data derived from the Web Data Commons project, focusing on:
- **T2Dv2 Gold Standard for Matching Web Tables to DBpedia**: 108 tables from 9 entity classes. [Access here](https://webdatacommons.org/webtables/goldstandardV2.html).
- **Schema.org Table Corpus 2023**: 92 tables from 8 entity classes. [Access here](https://webdatacommons.org/structureddata/schemaorgtables/2023/index.html#toc3).
#### Input Dataset
The input dataset consists of structured data with various attributes. Below is a glimpse of the top 5 rows, showcasing the structure and type of data we are dealing with:

| Rank | Title                                | Category         | Publisher |
|------|--------------------------------------|------------------|-----------|
| 1    | Super Smash Bros. Melee              | Fighting         | Nintendo  |
| 2    | Pikmin 2                             | Strategy/Sim     | Nintendo  |
| 3    | Legend of Zelda: Collector's Edition | RPG              | Nintendo  |
| 4    | Legend of Zelda: The Wind Waker      | Action Adventure | Nintendo  |
| 5    | Metal Gear Solid: Twin Snakes        | Action Adventure | Konami    |



D3L utilizes a comprehensive approach based on:

1. **Attribute Header Similarity**
2. **Value Similarity**
3. **Format Similarity**
4. **Value Distribution**
5. **Attribute value embeddings**
#### Output Datasets: Top k searched dataset results



In [ ]:
## Download required words
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

##### Generate LSH indexes for all evidence in D3L

In [4]:
from Utils import mkdir
# import and initialize D3L
from d3l.indexing.similarity_indexes import NameIndex, FormatIndex, ValueIndex, EmbeddingIndex, DistributionIndex
from d3l.input_output.dataloaders import CSVDataLoader
from d3l.querying.query_engine import QueryEngine
from d3l.utils.functions import pickle_python_object, unpickle_python_object
import os
#

data_path = "Datasets"
result_path = "Result/"
threshold = 0.5
mkdir(result_path)
#  collection of tables

dataloader = CSVDataLoader(
        root_path=data_path,
        encoding='utf-8'
)


##### Generating/loading NameIndex of tables
Name index: Use q-gram analysis of attribute names to calculate the Jaccard distance between their qsets.

In [ ]:
name_lsh = os.path.join(result_path, 'Name.lsh')
print(name_lsh)
if os.path.isfile(name_lsh):
    name_index = unpickle_python_object(name_lsh)
    print("Name LSH index: LOADED!")
else:
    name_index = NameIndex(dataloader=dataloader, index_similarity_threshold=threshold)
    pickle_python_object(name_index, name_lsh)
    print("Name LSH index: SAVED!")

##### Generating/loading FormatIndex of tables
 Format Index: Identifies data formats through regular expressions

In [ ]:
format_lsh = os.path.join(result_path, './format.lsh')
if os.path.isfile(format_lsh):
    format_index = unpickle_python_object(format_lsh)
    print("Format LSH index: LOADED!")
else:
    format_index = FormatIndex(dataloader=dataloader, index_similarity_threshold=threshold)
    pickle_python_object(format_index, format_lsh)
    print("Format LSH index: SAVED!")

##### Generating/loading ValueIndex of tables
Value Index: Employs TFIDF tokens to represent values, with Jaccard distance between their t-sets assessing similarity.

In [ ]:
value_lsh = os.path.join(result_path, './value.lsh')
if os.path.isfile(value_lsh):
    value_index = unpickle_python_object(value_lsh)
    print("Value LSH index: LOADED!")
else:
    value_index = ValueIndex(dataloader=dataloader, index_similarity_threshold=threshold)
    pickle_python_object(value_index, value_lsh)
    print("Value LSH index: SAVED!")

##### Generating/loading DistributionIndex of tables
Distribution Index: Assesses numeric attribute value relatedness via the Kolmogorov-Smirnov statistic, offering insights into domain-originating samples.

In [ ]:
   # DistributionIndex
distribution_lsh = os.path.join(result_path, './distribution.lsh')
if os.path.isfile(distribution_lsh):
    distribution_index = unpickle_python_object(distribution_lsh)
    print("Distribution LSH index: LOADED!")
else:
    distribution_index = DistributionIndex(dataloader=dataloader, index_similarity_threshold=threshold)
    pickle_python_object(distribution_index, distribution_lsh)
    print("Distribution LSH index: SAVED!")

##### Generating/loading EmbeddingIndex of tables
Embedding index: Determines textual content relatedness through cosine distance of their vector representations.

In [ ]:
embed_name = './embedding.lsh'
embedding_lsh = os.path.join(result_path, embed_name)
if os.path.isfile(embedding_lsh):
    embedding_index = unpickle_python_object(embedding_lsh)
    print("Embedding LSH index: LOADED!")
else:
    embedding_index = EmbeddingIndex(dataloader=dataloader,
                                     index_similarity_threshold=threshold)
    pickle_python_object(embedding_index, embedding_lsh)
    print("Embedding LSH index: SAVED!")


##### show the input table

In [ ]:
import pandas as pd
searched_table = 'T2DV2_122'
table_df = dataloader.read_table(searched_table)
print(table_df.head(5))
df = pd.read_csv("groundTruth.csv")

Query table in the framework using all the above indexes

In [ ]:
#Searched results, K =10
qe = QueryEngine(name_index, value_index, embedding_index, format_index,  distribution_index)
results, extended_results = qe.table_query(table=dataloader.read_table(table_name=searched_table),
                                           aggregator=None, k=10, verbose=True)

##### Output the results and check if the output tables have the same type as the input query table

In [ ]:
# Summarize searched results in a table
class_input_table = df[df['fileName'] == searched_table+".csv"]['class'].tolist()[0]

data = []
exceptions = []
average = []
for table, score in results:
        class_table = df[df['fileName'] == table+".csv"]['class'].tolist()[0]
        data.append((table, score,class_table))
        average.append(sum(score)/len(score))
        if class_table!=class_input_table:
            exceptions.append(table)
# Creating the DataFrame
result_summarization = pd.DataFrame(data, columns=["Table Name", "Scores", "Ground Truth Class"])
result_summarization = pd.concat([result_summarization.drop(["Scores"], axis=1), result_summarization["Scores"].apply(pd.Series)], axis=1).round(3)
result_summarization.columns = ["Table Name", "Class", "Header Score", "Value Score", "Embedding Score","Format Score",  "Distribution Score"]
result_summarization["average score"] = average
print(result_summarization)
print(result_summarization["Table Name"])

##### For tables that does not belong to the same class of input table, show the specific table.

In [ ]:
for table_name in exceptions:
    table_except = dataloader.read_table(table_name)
    table_except_part = table_except.head(5)
    print(table_except_part)

##### Individual search using different methods

In [ ]:
#Individual search results
# Name index query
topk = 10
def remove_search_col(listA, check_col):
    return [i for i, score in listA if i!=check_col]
        
def check_column(Dataloader:CSVDataLoader, combined_column_name):
    table_name, column_name = combined_column_name.split(".")
    table = Dataloader.read_table(table_name)
    return table[column_name]

def table_results(list_result):
    return pd.DataFrame(list_result, columns=["Column Name", "Scores"])

name_results = name_index.query(query="Title", k=topk)
print(f"Name results are \n {table_results(name_results)} \n")

# Value index query
value_results = value_index.query(query=table_df["Title"], k=topk)
columns = [check_column(dataloader, column) for column,score in value_results if column !="T2DV2_122.Title" ]
print(f"Value indexes results are \n {table_results(value_results)}\n")
print(f"example results searching Attribute value indexes:\n {columns[0]} \n")

# Embeddings index query
embedding_results = embedding_index.query(query=table_df["Title"], k=topk)
print(f"Embedding indexes results are \n{table_results(embedding_results)} \n")
embedding_column  = [check_column(dataloader, column) for column,score in embedding_results if column !="T2DV2_122.Title" ]
print(f"example results searching embedding value indexes:\n {embedding_column[0]} \n")


## Part 2: Dataset Navigation
### Framework Overview -- Aurum
This is a simplified version of Aurum. It includes two phases: signature building stage and relationship building stage.
Signatures: LSH indexes from D3L: name index and value index
Relationship Building Stage: Search similar columns based on similarity in name and value LSH indexes.


#### Prerequisites: detect subject columns and type of columns

In [ ]:
import pickle
from TableMiner.SCDection.TableAnnotation import TableColumnAnnotation as TA
"""
Find the column type and Named entity scores in each table,
 store the table and related column type/NE-scores info as dict in pickle file
"""
def subjectColDetection(DATA_PATH,RESULT_PATH):
    table_dict = {}
    if "dict.pkl" in os.listdir(RESULT_PATH):
        with open(os.path.join(RESULT_PATH,"dict.pkl"), "rb") as f:
            table_dict = pickle.load(f)
    else:
        table_names = os.listdir(DATA_PATH)
        for tableName in table_names:
            table_dict[tableName] = []
            table_ori = pd.read_csv(f"Datasets/{tableName}")
            try:
                annotation_table = TA(table_ori, SearchingWeb = False)
                annotation_table.subcol_Tjs()
                table_dict[tableName].append(annotation_table.annotation)
                table_dict[tableName].append(annotation_table.column_score)
            except:
                continue
        with open(os.path.join(RESULT_PATH,"dict.pkl"), "wb") as save_file:
            pickle.dump(table_dict, save_file)
    return table_dict
SubjectCol_dict = subjectColDetection(data_path,"Result")

##### Find the subject columns of result tables from Part I Dataset search.

In [ ]:
result_tables = df[df['class'] == class_input_table]['fileName'].tolist()
subject_columns=[]
all_columns = []
"""
Use iteration and the above column info dict to find the subject columns (and all columns)
 in each table.
"""
for table in result_tables:
    df_table = dataloader.read_table(table[:-4])
    annotation, NE_column_score = SubjectCol_dict[table]
    max_score = max(NE_column_score.values())
    all_columns.extend([f"{table[:-4]}.{df_table.columns[i]}" for i in NE_column_score.keys()])
    subcol_index = [key for key, value in NE_column_score.items() if value == max_score]
    for index in subcol_index:
        subject_columns.append(f"{table[:-4]}.{df_table.columns[index]}")
print(subject_columns)


In [ ]:
from Aurum.graph import buildGraph,draw_interactive_network
# Use Aurum to build the graph
G = buildGraph(dataloader, data_path, [name_index, value_index],target_path="Result", table_dict=SubjectCol_dict)
import networkx as nx
"""
Find the subgraph in the Aurum that contains the provided nodes and all the nodes that
have routine to these nodes
"""
def subgraph(given_nodes,graph:nx.Graph()):
    # Find the connected components containing the given node
    subgraphs = list(nx.connected_components(graph))
    relevant_nodes = set()
    for node in given_nodes:
        for sg in subgraphs:
            if node in sg:
                relevant_nodes.update(sg)
    new_graph = G.subgraph(relevant_nodes).copy()
    return new_graph
subject_columns_graph = subgraph(subject_columns, G)
result_SC_graph = subgraph(subject_columns, G)
draw_interactive_network(result_SC_graph)


In [ ]:
# See all columns in the graph
result_graph = subgraph(all_columns, G)
draw_interactive_network(result_graph)

## Part 3: Dataset Annotation
### Framework Overview -- TableMiner+
#### Input dataset: 13 tables from 13 domain, while each domain has 1 table
The 13 domains include:
1. **Airport**
2. **City**
3. **CollegeOrUniversity**
4. **Company**
5. **Continent**
6. **Country**
7. **Hospital**
8. **LandmarksOrHistoricalBuildings**
9. **Monarch**
10. **Movie**
11. **Museum**
12. **Scientist**
13. **VideoGame**

TableMiner+ has 4 steps:
1. **Subject Column Detection: Including column (data) type detection** 
2. **NE-Column interpretation - the LEARNING phase:**
***2.1 preliminary cell annotation***
***2.2 column semantic type annotation***
***2.3 property annotation***
3. **NE-Column interpretation - the UPDATE phase: revise annotation until all annotation is stabilized**
4. **Relation enumeration and annotating literal-columns(not included yet)**

##### show the example annotation table

In [ ]:
import pandas as pd
from TableMiner.LearningPhase.Update import TableLearning,  updatePhase
### The 13 tables from 13 different domains in the dataset
table_domains = ["Test_corpus_105", "T2DV2_168",
                 "Test_corpus_72","T2DV2_22",
                 "Test_corpus_90","T2DV2_155",
                 "T2DV2_191","Test_corpus_135",
                 "T2DV2_30","T2DV2_162",
                 "T2DV2_109","T2DV2_121"]

##### Perform NE-Column interpretation (Table Learning includes the process of subject column detection of a table)

In [ ]:

def table_annotation(tableName, subcol_dict):
    tableD = dataloader.read_table(tableName)
    print(tableD)
    annotation_table, NE_Score = subcol_dict[tableName + ".csv"]
    print(annotation_table)
    ### Learning phase of TableMiner+
    tableLearning = TableLearning(tableD, NE_column=NE_Score)
    ### Perform NE-Column interpretation - the UPDATE phase
    tableLearning.table_learning()
    updatePhase(tableLearning)
    return tableLearning

In [ ]:
def store_learning(table, learning, dict_path, dict_name):
    target_file = os.path.join(dict_path, dict_name)
    if os.path.isfile(target_file):
        with open(target_file, 'rb') as file:
            dict_annotation = pickle.load(file)
    else:
        dict_annotation = {}
    dict_annotation[table] = learning
    with open(target_file, 'wb') as file:
        pickle.dump(dict_annotation, file)


In [ ]:
searched_table = table_domains[0]
learning = table_annotation(searched_table, SubjectCol_dict)
store_learning(searched_table, learning, "Result", "annotationDict.pkl")

##### check the annotation of column and column header

In [ ]:
annotation_class = learning.get_annotation_class()
for column_index, learning_class in annotation_class.items():
    column = table_df.iloc[:,column_index]
    cellAnnotation  = learning_class.get_cell_annotation()
    ColumnSemantics = learning_class.get_winning_concepts()
    df = pd.concat([column, cellAnnotation], axis=1)
    print(f"column and Cell annotation of the column:\n{df}\n")
    print(f"Column {column.name} semantic type: {ColumnSemantics}")

## Part 4: Schema Inference
### Framework Overview -- Starmie
#### Input dataset: all 200 tables covering 13 specific domains
Embedding methods: Starmie
Table class inference method: Hierarchical clustering
Smilairity metric: Average column embeddings of each table
Type of result table clusters: the most frequently appeared class in the ground truth in each cluster


##### Use clustering on table's embeddings to detect the types/domains' of tables

In [ ]:
import clustering  as c
"""
Generation of Embeddings: Please run the ./starmie/cmd.sh script to generate the tables' embeddings 
We uploaded the sample embeddings generated by this script to the Result path.
"""
clustering_result = c.typeInference("Result/tableEmbeddings.pkl", "Agglomerative", numEstimate=13)
index_cluster = [i for i, tables in clustering_result.items() if "T2DV2_122.csv" in tables]
# The overall result of clustering in a dataframe, includes cluster id, GT label, size, precision,
# Ranked by precision
result_precision = c.result_precision(clustering_result)
print(result_precision)

In [ ]:
# Check the last second clusters' score
print("\n",result_precision.iloc[-2])
# Check inside the cluster, what kind of table does it contain
checked_cluster = clustering_result[result_precision.iloc[-2]["cluster id"]]
innerInfo = c.inner_cluster(checked_cluster)
print(innerInfo)
# select random two tables inside the sample cluster; check and compare their column headers
# to find the difference/similarities between then
rows = c.sample_tables_cluster(innerInfo)
first_table = dataloader.read_table(rows.iloc[0, 0][:-4])
second_table = dataloader.read_table(rows.iloc[1, 0][:-4])
print(first_table.columns,"\n",second_table.columns)

##### Use clustering on column embeddings to detect column's type
column type inference: using hierarchical clustering on the column embeddings


In [ ]:
import os
### Cluster all column embeddings under specified domain
column_clustering = c.conceptualAttri(os.path.join(c.current_dir_path, "Datasets"),
                os.path.join(c.current_dir_path, "Result/tableEmbeddings.pkl"),
                clustering_method="Agglomerative",
                domain="VideoGame",
                numEstimate=13)

check_table = "T2DV2_122"

for column_index, column_clusters in column_clustering.items():
    for i in  column_clusters:
        if check_table in i:
            column = [i for i in column_clusters if check_table in i][0]
            print(f"Cluster has {column} \n",column_clusters, "\n")
